In [1]:
print("thinking...")
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import numpy as np
print("thinking...")
from matplotlib import pyplot as plt
from ipywidgets import interact, IntSlider
print("thinking...")
from ipywidgets import FloatSlider
from ipywidgets import Layout 
from scipy import signal
from matplotlib import gridspec
from scipy.fft import fftshift
import warnings
import os
print("\n------------------------------ A GUI icon shoud've appear in your task bar ------------------------------")
print("---------------------- Select that to pick the data file you would like to analyze ----------------------\n")


################################################################################################################################
################################################################################################################################


def csv_upload():
    
    file_path_global = None ## Variable for data path 

    def upload_file():
        global file_path_global ## makes variable global 


        file_path_global = filedialog.askopenfilename(title="Select a file", filetypes=[("CSV Files", "*.csv")])

        print(f"\nFile selected: {file_path_global}")
        print("\n---------------------------------------Close the GUI program window---------------------------------------")

    root = tk.Tk() ## Create the program window
    root.title("Multi File Uploader")

    upload_button = tk.Button(root, text="Upload Sensor Data", command=upload_file) ## Create buttons for uploading files
    upload_button.pack(pady=20)

    root.mainloop() ## Runs the application
    
    print("thinking...")

#------------------------------------------------- Selects info from metadata -------------------------------------------------#


    metadata_rows = []
    metadata = {}

    with open(file_path_global, 'r') as f:
        for _ in range(5):
            metadata_rows.append(f.readline().strip()) 

    for row in metadata_rows:
        if ':' in row: 
            key, value = row.split(':', 1)
            metadata[key.strip()] = value.strip()


#-----------------------------------------# This is the sample rate from the metadata #----------------------------------------#

    sample_rate = float(metadata["Sample Rate"])
    print("\nSample Rate: " + metadata["Sample Rate"]) #------------ Prints out the sample rate as a sanity check -------------#

#------------------------------------------------------------------------------------------------------------------------------#

    print("thinking...")

    if function == "mag":
        calibration = (1.222e-05)/21
        ctrl_calibration = (1.222e-05)/21
        control_file = 'mag_data.csv'
        
    elif function == "seis":
        calibration = (0.0125e-1)/21
        ctrl_calibration = 1.222e-05
        control_file = 'ligo_seis_data.txt'
        
    elif function == "minimus":
        calibration = 0.0015e-6
        ctrl_calibration = 1.222e-05
        control_file = 'ligo_seis_data.txt'
    
#------------------------------------------------------------------------------------------------------------------------------#
    
    seis = pd.read_csv(file_path_global, skiprows = 5, delimiter=',') 

    seis.columns = ["Sample", "Time (s)", "Noise (V)", "Channel E (V)", "Channel N (V)", "Channel Z (V)", "blank"]

    #sam = seis['Sample']
    tim = seis['Time (s)']
    noi = seis['Noise (V)'] * (0.0125e-1)/21
    z = seis['Channel Z (V)'] * (0.0125e-1)/21 #-------------- This is the conversion factor for Volts to Tesla ---------------#
    n = seis['Channel N (V)'] * (0.0125e-1)/21 
    e = seis['Channel E (V)'] * (0.0125e-1)/21


#------------------------------------------------------------------------------------------------------------------------------#


    script_dir = os.getcwd()
    control_data_path = os.path.join(script_dir, 'ligo_seis_data.txt')
    
    print("thinking...")

    if control_file == 'ligo_seis_data.txt':
        
        if os.path.exists(control_data_path):
            huddle = pd.read_csv(control_data_path, delimiter="\s+")

            huddle.columns = ["frequency", "x", "y", "z"]

            hf = huddle["frequency"]
            hx = huddle["x"] * ctrl_calibration
            hy = huddle["y"] * ctrl_calibration
            hz = huddle["z"] * ctrl_calibration

        else:
            print("LIGO data not found\n")
            
    elif control_file == 'mag_data.csv':
        ligo = pd.read_csv(control_data_path, delimiter=",", skiprows=5)
        #ligo.columns = ["sample","noise", "x", "y","z"]

        ck = ligo['Voltage Z (V)'] * (1.222e-05)/21  #[0:647487] 
        cj = ligo['Voltage Y (V)'] * (1.222e-05)/21
        ci = ligo['Voltage X (V)'] * (1.222e-05)/21
        csr = 5120
    
else:
    print("Control data not found\n")
        
        
    return 
    
    
################################################################################################################################
#------------------------------------------------------------------------------------------------------------------------------#
################################################################################################################################


def get_optional_float(prompt, func, ID, allow_back=True):
    """
    Prompts user for a float or 'none'. Optionally allows an 'x' keyword to return control.
    Returns float, None, or 'BACK' if requested.
    """
    while True:
        user_input = input(prompt).strip().lower()
        
        
#------------------------------------------------------------------------------------------------------------------------------#

        if func == "time_series":   
            if user_input in ["none", "", "default"]:
                return None
            if allow_back and user_input == "x":
                return "BACK"
            try:
                return float(user_input)
            except ValueError:
                print("Invalid input. Please enter a number, 'none', or 'x'.")        
            
#------------------------------------------------------------------------------------------------------------------------------#
            
        elif func == "asd":
            if ID == "fft_len":
                if user_input in ["none", "", "default"]:
                    return 128
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")

            elif ID == "overlap": 
                if user_input in ["none", "", "default"]:
                    return 50
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
            elif ID == "my_max": 
                if user_input in ["", "default"]:
                    return 10e-7
                elif user_input in ["none"]:
                    return None
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
            elif ID == "my_min": 
                if user_input in ["", "default"]:
                    return 10e-14
                elif user_input in ["none"]:
                    return None
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
            elif ID == "y_max": 
                if user_input in ["", "default"]:
                    return 10e-6
                elif user_input in ["none"]:
                    return None
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
            elif ID == "y_min": 
                if user_input in ["", "default"]:
                    return 50e-12
                elif user_input in ["none"]:
                    return None
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
            elif ID == "x_max": 
                if user_input in ["", "default"]:
                    return 100
                elif user_input in ["none"]:
                    return None
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
            elif ID == "x_min": 
                if user_input in ["", "default"]:
                    return 0.1
                elif user_input in ["none"]:
                    return None
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
            elif ID == "prominence": 
                if user_input in ["", "default"]:
                    return 5
                elif user_input in ["none"]:
                    return None
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
#------------------------------------------------------------------------------------------------------------------------------#

        elif func == "spectrogram":
    
            if ID == "fft_len":
                if user_input in ["none", "", "default"]:
                    return 10
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
        
            elif ID == "overlap": 
                if user_input in ["none", "", "default"]:
                    return 50
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
            
            elif ID == "y_max": 
                if user_input in ["", "default"]:
                    return 100
                elif user_input in ["none"]:
                    return None
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
            elif ID == "y_min": 
                if user_input in ["none", "", "default"]:
                    return 0
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
            elif ID == "x_max": 
                if user_input in ["none", "", "default"]:
                    return None
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
            elif ID == "x_min": 
                if user_input in ["none","", "default"]:
                    return None
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
            elif ID == "c_min": 
                if user_input in ["", "default"]:
                    return 10e-10
                elif user_input in ["none"]:
                    return None
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
            elif ID == "c_max": 
                if user_input in ["", "default"]:
                    return 10e-9
                elif user_input in ["none"]:
                    return None
                if allow_back and user_input == "x":
                    return "BACK"
                try:
                    return float(user_input)
                except ValueError:
                    print("Invalid input. Please enter a number, 'none', 'default', or 'x'.")
                    
#------------------------------------------------------------------------------------------------------------------------------#

            else:
                print("Carlos didn't do his job properly")
    

    
'''##########################################################################################################################'''
'''--------------------------------------------------------------------------------------------------------------------------'''
'''##########################################################################################################################'''

def plot_time_series():
    #print("Running time series plot...")
    
    def time_series(channels,time, zed, north, east, xmax, xmin, ymax, ymin):
        plt.figure(figsize = (19, 11))
        ax = plt.gca()
        ax.yaxis.get_offset_text().set_fontsize(16)
        
        if channels == "1":
            plt.plot(time, z, linewidth = 1.5, color = 'black', label = 'Z') 
            plt.plot(time, n, linewidth = 1.5, color = 'red', label = 'N')
            plt.plot(time, e, linewidth = 1.5, color = 'mediumblue', label = 'E')
            
        elif channels == "2":
            plt.plot(time, north, linewidth = 1.5, color = 'red', label = 'N')
            
        elif channels == "3":
            plt.plot(time, east, linewidth = 1.5, color = 'mediumblue', label = 'E')
            
        elif channels == "4":
            plt.plot(time, zed, linewidth = 1.5, color = 'black', label = 'Z')

        plt.legend(loc = "upper right", fontsize = 18)
        plt.title("Seismic Data Time Series", fontweight = 'bold', fontsize = 25)
        plt.xlabel("Time (s)", fontweight = "bold", fontsize = 20)
        plt.ylabel("Amplitude (m/s)", fontweight = "bold", fontsize = 20)

        plt.xlim(0, tim[len(tim)-1])

        plt.yticks(fontsize = 20, fontweight = "bold")
        plt.xticks(fontsize = 20, fontweight = "bold")
        plt.grid(True)


        plt.show()
        
    time_series("1", tim, z, n, e, None, None, None, None)
    
#------------------------------------------------------------------------------------------------------------------------------#

    while True:
        '''print("\n---------------------------------------")
        print("-- Time Series Options --")
        #print("1: Save plot to file")
        print("1: Zoom in on specific time range")
        print("x: Return to main menu")
        print("---------------------------------------")'''

        sub_choice = input("\n\n---------------------------------------\n1: Zoom in on specific time range\nx: Return to main menu\n---------------------------------------\n\nEnter your choice (1-3): ")
        
        if sub_choice == "1":
            print("\nWhat x limits do you want (You can input None)")
            x_max = get_optional_float("Upper bound: ", "time_series", None)
            if x_max == "BACK":
                return
            
            x_min = get_optional_float("Lower bound: ", "time_series", None)
            if x_min == "BACK":
                return
            
            print("\nWhat y limits do you want (You can input None)")
            y_max = get_optional_float("Upper bound: ", "time_series", None)
            if y_max == "BACK":
                return
            
            y_min = get_optional_float("Lower bound: ", "time_series", None)
            if y_min == "BACK":
                return
                            
#------------------------------------------------------------------------------------------------------------------------------#

            while True:
        
                ''' This is what the menu will look like printed out. It iss done in one input line to ensure that the prompt 
                and input line are outputted together
                
                ---------------------------------------
                "Do you want all three channels or just one?
                "1: All three
                "2: North
                3: East
                4: Z
                x: Return to Time Series Options
                ---------------------------------------'''
                
                channel = input("\n\n---------------------------------------\nDo you want all three channels or just one?\n1: All three\n2: North\n3: East\n4: Z\nx: Return to Time Series Options\n---------------------------------------\n\nEnter your choice (1-5): ")

#------------------------------------------------------------------------------------------------------------------------------#

                if channel == "1": ## All Channels
                    time_series("1", tim, z, n, e, x_max, x_min, y_max, y_min)
                    

                elif channel == "2": ## N Channel
                    time_series("2", tim, z, n, e, x_max, x_min, y_max, y_min)
                    

                elif channel == "3": ## E Channel
                    time_series("3", tim, z, n, e, x_max, x_min, y_max, y_min)
                    

                elif channel == "4": ## Z Channel
                    time_series("4", tim, z, n, e, x_max, x_min, y_max, y_min)
                    

                elif channel == "x" or channel == "":
                    print("\nReturning to Time Series Options...")
                    break
                    
                else:
                    print("\nInvalid input. Try again.")
    
#------------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------------#

        elif sub_choice == "x" or sub_choice == "" or sub_choice == "2":
            print("\nReturning to main menu...")
            break
        else:
            print("\nInvalid input. Try again.")    

    
'''##########################################################################################################################'''
'''--------------------------------------------------------------------------------------------------------------------------'''
'''##########################################################################################################################'''

def plot_spectrum():
    print("Plotting Spectrum...")
    
#-------------------------------------------------------- Plot limits ---------------------------------------------------------#
    x_max = 100 ## in terms of frequency
    x_min = 0.1

#----------------------------------------------------- Velocity Y limits ------------------------------------------------------#
    y_max = 10e-6 ## in terms of ms⁻¹/√Hz
    y_min = 50e-12  

#--------------------------------------------------- Displacement Y Limits ----------------------------------------------------#
    my_max = 10e-7 ## in terms of m/√Hz
    my_min = 10e-14

#-------------------------------------------------------- fft length ----------------------------------------------------------#
    fft_length = 128 ## in terms of seconds

#---------------------------------------------------- Precent FFT Overlap -----------------------------------------------------#
    overlap = 50 ## 50% fft overlap
    
    
    
################################################################################################################################
#----------------------------------------- The PSD is calculated via welch from scipy -----------------------------------------#
#------------------------------------------------------------------------------------------------------------------------------#
    
    
    warnings.simplefilter('ignore')
    
    
    def asd(data_z, data_n, data_e, sample_rate, lf, lx, ly, lz, over_lap, fft, signal_prom, ymin, ymax, xmin, xmax, func, channel):
#----------# If you don't want to look at the whole time series, you can add '1' to the variables, i.e z1, y1, x1 #------------#

        f_z, Pxx_den_z = signal.welch(data_z, sample_rate, window= 'hamming', 
                                      nperseg= (sample_rate *fft), noverlap= round(sample_rate *(overlap *0.01)) )
        f_n, Pxx_den_n = signal.welch(data_n, sample_rate, window= 'hamming', 
                                      nperseg= (sample_rate *fft), noverlap= round(sample_rate *(overlap *0.01)) )
        f_e, Pxx_den_e = signal.welch(data_e, sample_rate, window= 'hamming', 
                                      nperseg= (sample_rate *fft), noverlap= round(sample_rate *(overlap *0.01)) )

#------------------ nfft is how long the fft bin lengths, noverlap is bin overlap, window is the window used ------------------#
#---------------------- We used 100 sec bins, 0.5 sec overlap, and hamming window, these can be changed -----------------------#
#------------------------------------------------ # DO NOT CHANGE/REMOVE 'fs' # -----------------------------------------------#


        az = np.sqrt(Pxx_den_z)
        an = np.sqrt(Pxx_den_n) #------------------------------ This turns the PSD into an ASD --------------------------------#
        ae = np.sqrt(Pxx_den_e)

        mz = az / (2*np.pi *f_z)
        mn = an / (2*np.pi *f_n) #------------------------------------ Converts m/s to m --------------------------------------#
        me = ae / (2*np.pi *f_e)
        
        log_z = np.log(az)
        log_n = np.log(an) #------------ The log of each ASD must be found in order to find the peaks/frequences --------------#
        log_e = np.log(ae)
        
        peak_z, _ = signal.find_peaks(log_z, 
                                      prominence = signal_prom) #---------- This is where the frequencies are found -----------#
        peak_n, _ = signal.find_peaks(log_n, 
                                      prominence = signal_prom) #----------- If some are missed or there are extra ------------#
        peak_e, _ = signal.find_peaks(log_e, 
                                      prominence = signal_prom) #------ you will need to change the prominence variable -------#


#------------------------------------------------------ PLots Spectra ---------------------------------------------------------#

        plt.figure(figsize = (20, 8))

        plt.yscale('log')
        plt.xscale('log')

#------------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------------#

        if func == "velocity":
            
            up = az
            north = an
            east = ae

            ylabel = "Amplitude [ms⁻¹/√Hz]"
            
            if channel == "all":
                
                title = "Seismic Velocity Data ASD"
                
                plt.plot(f_z, up, color = 'black', linewidth = 1.75, label = 'Z Direction')
                plt.plot(f_n, north, color = 'red', linewidth = 1.75, label = 'N Direction')
                plt.plot(f_e, east, color = 'mediumblue', linewidth = 1.75, label = 'E Direction')
                
                plt.scatter(f_z[peak_z], up[peak_z], s = 100, color = 'limegreen', marker = 'x', 
                    linewidths = 2.5, label = 'Signal Peaks')
                plt.scatter(f_n[peak_n], north[peak_n], s = 100, color = 'limegreen', marker = 'x', 
                            linewidths = 2.5)
                plt.scatter(f_e[peak_e], east[peak_e], s = 100, color = 'limegreen', marker = 'x', 
                            linewidths = 2.5)
                
            elif channel == "north":
                plt.plot(f_n, north, color = 'red', linewidth = 1.75, label = 'N Direction', alpha = 0.5)
                plt.plot(lf,ly, color = "dimgrey", label = "LIGO Washington Y", alpha = 0.5)
                title = "N Direction"
            
            elif channel == "east":
                plt.plot(f_e, east, color = 'mediumblue', linewidth = 1.75, label = 'E Direction', alpha = 0.5)
                plt.plot(lf,lx, color = "dimgrey", label = "LIGO Washington X", alpha = 0.5)
                title = "E Direction"

            elif channel == "zed":
                plt.plot(f_z, up, color = 'black', linewidth = 1.75, label = 'Z Direction', alpha = 0.5)
                plt.plot(lf,lz, color = "dimgrey", label = "LIGO Washington Z", alpha = 0.5)
                title = "Z Direction"

#------------------------------------------------------------------------------------------------------------------------------#

        elif func == "displacement":
            up = mz
            north = mn
            east = me

            ylabel = "Amplitude [m/√Hz]"
            title = "Seismic Dispacement Data ASD"
            
            plt.plot(f_z, up, color = 'black', linewidth = 1.75, label = 'Z Direction')
            plt.plot(f_n, north, color = 'red', linewidth = 1.75, label = 'N Direction')
            plt.plot(f_e, east, color = 'mediumblue', linewidth = 1.75, label = 'E Direction')
            
            plt.scatter(f_z[peak_z], up[peak_z], s = 100, color = 'limegreen', marker = 'x', 
                    linewidths = 2.5, label = 'Signal Peaks')
            plt.scatter(f_n[peak_n], north[peak_n], s = 100, color = 'limegreen', marker = 'x', 
                        linewidths = 2.5)
            plt.scatter(f_e[peak_e], east[peak_e], s = 100, color = 'limegreen', marker = 'x', 
                        linewidths = 2.5)

        else: 
            print("Carlos didn't do his job properly")

#------------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------------#

        plt.legend(loc = "lower left", fontsize = 14.5, ncol = 2)

        plt.title('FFT: ' + str(fft_length) + "s", fontsize = 20, loc = "left",style ='italic')
        plt.title("Overlap: " + str(overlap) + "%", fontsize = 20, loc = "right",style ='italic')

        plt.title(title, fontweight = 'bold', fontsize = 25)

        plt.xlabel("Frequency [Hz]", fontweight = "bold", fontsize = 20)
        plt.ylabel(ylabel, fontweight = "bold", fontsize = 20)

        plt.yticks(fontsize = 20, fontweight = "bold")
        plt.xticks(fontsize = 20, fontweight = "bold")

        plt.ylim(y_min,y_max)
        plt.xlim(x_min,x_max)

        plt.grid(True, which="both", ls="-")
        plt.show()


#----------------------------------------------------- Plots Velocity ---------------------------------------------------------#
    
    asd(z, n, e, sr, None, None, None, None, overlap, fft_length, 5, y_min, y_max, x_min, x_max, "velocity", "all")


#---------------------------------------------------- Plots Displacement ------------------------------------------------------#

    asd(z, n, e, sr, None, None, None, None, overlap, fft_length, 5, my_min, my_max, x_min, x_max, "displacement", "all")

#------------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------------#
    
    while True:
        '''print("\n\n---------------------------------------")
        print("-- ASD Spectra Options --")
        print("1: Compare to LIGO Hanford Data")
        print("2: Change plot limits / FFT Parameters")
        print("x: Return to main menu")
        print("---------------------------------------")'''

        sub_choice = input("\n\n---------------------------------------\n-- ASD Spectra Options --\n1: Compare to LIGO Hanford Data\n2: Change plot limits / FFT Parameters\nx: Return to main menu\n---------------------------------------\n\nEnter your choice (1-3): ")
        
        if sub_choice == "1":
        
#---------------------------------------------------------- E Channel ---------------------------------------------------------#

            asd(z, n, e, sr, hf, hx, hy, hz, overlap, fft_length, 5, y_min, y_max, x_min, x_max, "velocity", "east")
    
#---------------------------------------------------------- N Channel ---------------------------------------------------------#

            asd(z, n, e, sr, hf, hx, hy, hz, overlap, fft_length, 5, y_min, y_max, x_min, x_max, "velocity", "north")
    
#---------------------------------------------------------- Z Channel ---------------------------------------------------------#

            asd(z, n, e, sr, hf, hx, hy, hz, overlap, fft_length, 5, y_min, y_max, x_min, x_max, "velocity", "zed")
        
        
#------------------------------------------------------------------------------------------------------------------------------#            

        if sub_choice == "2":
            
            print("\nWhat x limits do you want (You can input None)")
            x_max = get_optional_float("Upper bound: ", "asd", "x_max")
            if x_max == "BACK":
                return
            
            x_min = get_optional_float("Lower bound: ", "asd", "x_min")
            if x_min == "BACK":
                return
                        
#------------------------------------------------------------------------------------------------------------------------------#

            print("\nWhat y limits do you want for the velocity plot? (You can input None or default)")
            y_max = get_optional_float("Upper bound: ", "asd", "y_max")
            if y_max == "BACK":
                return
            
            y_min = get_optional_float("Lower bound: ", "asd", "y_min")
            if y_min == "BACK":
                return
                
            print("\nWhat y limits do you want for the displacement plot? (You can input None or Default)")
            my_max = get_optional_float("Upper bound: ", "asd", "my_max")
            if my_max == "BACK":
                return
            
            my_min = get_optional_float("Lower bound: ", "asd", "my_min")
            if my_min == "BACK":
                return
            
#------------------------------------------------------------------------------------------------------------------------------#            
  
            print("\nHow long do you want the FFTs to be? (You can input None)")
            fft_length = get_optional_float("FFT length [s]: ", "asd", "fft_len")
            if fft_length == "BACK":
                return
            
            print("\nHow much FFT overlap do you want? i.e 50 for 50%")
            overlap = get_optional_float("Overlap Percentage: ", "asd", "overlap")
            if overlap == "BACK":
                return
            
            print("\nWhat prominence level do you want for peaks? (Default is 5)")
            prom = get_optional_float("Prominence level: ", "asd", "prominence")
            if prom == "BACK":
                return
                
#----------------------------------------------------- Plots Velocity ---------------------------------------------------------#
    
            asd(z, n, e, sr, None, None, None, None, overlap, fft_length, prom, y_min, y_max, x_min, x_max, "velocity", "all")


#---------------------------------------------------- Plots Displacement ------------------------------------------------------#

            asd(z, n, e, sr, None, None, None, None, overlap, fft_length, prom, my_min, my_max, x_min, x_max, "displacement", "all")

#------------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------------#
    
        elif sub_choice == "x" or sub_choice == "" or sub_choice == "3":
            print("\nReturning to main menu...")
            break
        else:
            print("\nInvalid input. Try again.") 
    
    
'''##########################################################################################################################'''
'''--------------------------------------------------------------------------------------------------------------------------'''
'''##########################################################################################################################'''


def plot_spectrogram():
    print("\nPlotting Spectrogram...\n\nThis may take a while")
    
    
#------------------------------------------------------ Colorbar Limits -------------------------------------------------------#
    cmin = 10e-10
    cmax = 10e-9

#-------------------------------------------------------- Plot limits ---------------------------------------------------------#
    y_max = 100 ## In terms of frequency
    y_min = 0

#-------------------------------------------------------- fft length ----------------------------------------------------------#
    fft_len = 10 ## in terms of seconds

#---------------------------------------------------- Precent FFT Overlap -----------------------------------------------------#
    overlap = 50 ## 50% fft overlap
    
#------------------------------------------------------------------------------------------------------------------------------#
    
    
    
################################################################################################################################
#----------------------------# The spectrograms are calculated with signal.spectrogram from SciPy #----------------------------#
#------------------------------------------------------------------------------------------------------------------------------#


    def spectrogram(data, channel, sample_rate, over_lap, fft, c_min, c_max, ymin, ymax, xmin, xmax):
        warnings.simplefilter('ignore')

        f, t, Sxx = signal.spectrogram(data, sample_rate, window= 'hamming'
                                          , nperseg=round(sample_rate *fft), noverlap= round(sample_rate *(over_lap *0.01)) )

#------------------------------------------------------------------------------------------------------------------------------#

        if channel == "z":
            title = "Z Direction"
        elif channel == "e":
            title = "E Direction"
        elif channel == "n":
            title = "N Direction"
        else:
            print("Carlos didn't do his job properly")

#------------------------------------------------------------------------------------------------------------------------------#

        fig = plt.figure(figsize = (20,8))

        gs = gridspec.GridSpec(1,1)

        axis1 = fig.add_subplot(gs[0,0])

        m0 = axis1.pcolormesh(t, f, np.sqrt(Sxx), shading='gouraud', vmin = c_min, vmax = c_max)

        axis1.set_title('FFT: ' + str(fft) + "s", fontsize = 15, loc = "left",style ='italic')
        axis1.set_title("Overlap: " + str(over_lap) + "%", fontsize = 15, loc = "right",style ='italic')

        axis1.set_title(title, fontweight = 'bold' , fontsize = 25)
        axis1.set_xlabel('Time [s]', fontweight = 'bold' , fontsize = 20)
        axis1.set_ylabel('Frequency [Hz]', fontweight = 'bold' , fontsize = 20)
        axis1.tick_params(labelsize=16)

        cbar0 = fig.colorbar(m0, pad =0.02)
        cbar0.ax.tick_params(labelsize=16)
        cbar0.set_label(label='Intensity [ms⁻¹/√Hz]' ,weight='bold', fontsize = 16)
        cbar0.ax.yaxis.offsetText.set_fontsize(14)

        axis1.set_ylim(y_min,y_max) ## Uncomment to set limits
        axis1.set_xlim(x_min,x_max)
        plt.show()


#---------------------------------------------------------- E Channel ---------------------------------------------------------#
    spectrogram(e, "e", sr, overlap, fft_len, cmin, cmax, y_min, y_max, None, None)
    
#---------------------------------------------------------- N Channel ---------------------------------------------------------#
    spectrogram(n, "n", sr, overlap, fft_len, cmin, cmax, y_min, y_max, None, None)
    
#---------------------------------------------------------- Z Channel ---------------------------------------------------------#
    spectrogram(z, "z", sr, overlap, fft_len, cmin, cmax, y_min, y_max, None, None)

    
################################################################################################################################
#------------------------------------------------------------------------------------------------------------------------------#    
    
    while True:
        '''print("\n---------------------------------------")
        print("-- Spectrogram Options --")
        #print("1: Save plot to file")
        print("1: Change plot limits / FFT Parameters")
        print("x: Return to main menu")
        print("---------------------------------------")'''

        sub_choice = input("\n---------------------------------------\n-- Spectrogram Options --\n1: Change plot limits / FFT Parameters\nx: Return to main menu\n---------------------------------------\n\nEnter your choice (1-2): ")
        
#------------------------------------------------------------------------------------------------------------------------------#            

        if sub_choice == "1":
            
            print("\nWhat x limits do you want (You can input None)")
            x_max = get_optional_float("Upper bound: ", "spectrogram", "x_max")
            if x_max == "BACK":
                return
            
            x_min = get_optional_float("Lower bound: ", "spectrogram", "x_min")
            if x_min == "BACK":
                return

            print("\nWhat y limits do you want for the velocity plot? (You can input None or default)")
            y_max = get_optional_float("Upper bound: ", "spectrogram", "y_max")
            if y_max == "BACK":
                return
            
            y_min = get_optional_float("Lower bound: ", "spectrogram", "y_min")
            if y_min == "BACK":
                return
            
#------------------------------------------------------------------------------------------------------------------------------#
                
            print("\nWhat intensity limits do you want for the plot? (You can input None or Default)")
            cmax = get_optional_float("Upper bound: ", "spectrogram", "c_max")
            if cmax == "BACK":
                return
            
            cmin = get_optional_float("Lower bound: ", "spectrogram", "c_min")
            if cmin == "BACK":
                return
            
#------------------------------------------------------------------------------------------------------------------------------#            
  
            print("\nHow long do you want the FFTs to be? (You can input None)")
            fft_len = get_optional_float("FFT length [s]: ", "spectrogram", "fft_len")
            if fft_len == "BACK":
                return
            
            print("\nHow much FFT overlap do you want? i.e 50 for 50%")
            overlap = get_optional_float("Overlap Percentage: ", "spectrogram", "overlap")
            if olap == "BACK":
                return
            
            print("Replotting Spectrogram...\n\nThis may take a while")
            
            
#---------------------------------------------------------- E Channel ---------------------------------------------------------#
            spectrogram(e, "e", sr, overlap, fft_len, cmin, cmax, y_min, y_max, x_min, x_max)
    
#---------------------------------------------------------- N Channel ---------------------------------------------------------#
            spectrogram(n, "n", sr, overlap, fft_len, cmin, cmax, y_min, y_max, x_min, x_max)
    
#---------------------------------------------------------- Z Channel ---------------------------------------------------------#
            spectrogram(z, "z", sr, overlap, fft_len,  cmin, cmax, y_min, y_max, x_min, x_max)

    
#------------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------------#

        elif sub_choice == "x" or sub_choice == "":
            print("\nReturning to main menu...")
            break
        else:
            print("\nInvalid input. Try again.")

    
'''##########################################################################################################################'''
'''--------------------------------------------------------------------------------------------------------------------------'''
'''##########################################################################################################################'''


def main_menu():
    while True:
        print("\n---------------------------------------")
        print("=== Data Analysis Menu ===")
        print("1: Plot Time Series")
        print("2: Plot FFT Spectrum")
        print("3: Plot Spectrogram")
        print("x: Exit")
        print("-----------------------------------------")

        choice = input("\nEnter your choice (1-4): ")

        if choice == "1":
            plot_time_series()
        elif choice == "2":
            plot_spectrum()
        elif choice == "3":
            plot_spectrogram()
        elif choice == "x" or  choice =="":
            print("\nExiting. Goodbye!")
            break
        else:
            print("\nInvalid input. Please choose a number between 1 and 4.")

if __name__ == "__main__":
    main_menu()


thinking...
thinking...
thinking...

------------------------------ A GUI icon shoud've appear in your task bar ------------------------------
---------------------- Select that to pick the data file you would like to analyze ----------------------


File selected: C:/Users/cacam/Documents/data_files/10-01-compare/seismo_test_cal_2024-10-01T22-07-09-752.csv

---------------------------------------Close the GUI program window---------------------------------------

Sample Rate: 1651.6129032258063

---------------------------------------
=== Data Analysis Menu ===
1: Plot Time Series
2: Plot FFT Spectrum
3: Plot Spectrogram
x: Exit
-----------------------------------------

Enter your choice (1-4): 

Exiting. Goodbye!


In [ ]:
def main_menu():
    while True:
        print("\n---------------------------------------")
        print("1: Mag")
        print("2: Seis")
        print("x: Exit")
        print("\n---------------------------------------")
        
        choice = input("\nEnter your choice (1-4): ")
        
        if choice == "1":
            
            function = "Mag"
            
        
            while True:
                print("\n---------------------------------------")
                print("=== Magnetometer Analysis Menu ===")
                print("1: Plot Time Series")
                print("2: Plot FFT Spectrum")
                print("3: Plot Spectrogram")
                print("x: Return to main menu")
                print("-----------------------------------------")

                choice = input("\nEnter your choice (1-4): ")

                if choice == "1":
                    plot_time_series()
                elif choice == "2":
                    plot_spectrum()
                elif choice == "3":
                    plot_spectrogram()

                elif sub_choice == "x" or sub_choice == "":
                    print("\nReturning to main menu...")
                    break
                else:
                    print("\nInvalid input. Try again.")
                
        if choice == "2":
            while True:
                print("\n---------------------------------------")
                print("=== Seismometer Menu ===")
                print("1: WebDAQ Analysis")
                print("2: Minimus Analysis")
                print("x: Return to main menu")
                print("-----------------------------------------")
            
        elif choice == "x" or  choice =="":
            print("\nExiting. Goodbye!")
            break
        else:
            print("\nInvalid input. Please choose a number between 1 and 4.")

if __name__ == "__main__":
    main_menu()

In [ ]:
'''
#------------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------------#

        if sub_choice == "1":
            print("\nWhat you do you want to save the plot as?")
            print("No spaces or invalid chacaters (#, %, &, {, }, $, !, ', +, \``, =, *, :, <, >, ?, /, \, |)")
            name = input("plot name: ")

            print("\nWhat format would you like?")
            print("png, jpg, jpeg, pdf")
            Format = input("plot format: ")

            plt.savefig(str(name) + '.' + str(Format), bbox_inches='tight')
            print("The image has been saved to this directory (☆•ヮ•)八(•ヮ•)")


#------------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------------#
'''